In [99]:
# install.packages("bbmle")
# install.packages("readxl")
# install.packages("gridExtra")
# install.packages("ggplot2")
# install.packages("egg")

In [1]:
library("bbmle")
library("readxl")
library("gridExtra")
library("ggplot2")
library("egg")

Loading required package: stats4



In [2]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 200)

In [4]:
parental_effect_log_likelihood <- function(beta_m_0, beta_m_1, beta_p_0, beta_p_1, x) {
    # paternal_count, maternal_count, unphased_count, paternal_age, maternal_age, 
    paternal_count <- x$paternal_count
    maternal_count <- x$maternal_count
    unphased_count <- x$unphased_count
    paternal_age <- x$paternal_age
    maternal_age <- x$maternal_age

    maternal_reg_term = beta_m_0 + beta_m_1 * maternal_age
    paternal_reg_term = beta_p_0 + beta_p_1 * paternal_age
    if (any(maternal_reg_term <= 0)) {
        cat(beta_m_0, beta_m_1, maternal_reg_term)
    }
    return(-sum(maternal_count * log(maternal_reg_term) + paternal_count * log(paternal_reg_term) + unphased_count * log(maternal_reg_term + paternal_reg_term) - (maternal_reg_term + paternal_reg_term)))
}

estimate_paternal_maternal_effects <- function(df, beta_m_0_ini, beta_m_1_ini, beta_p_0_ini, beta_p_1_ini, print_output=FALSE) {
    m_ll <- mle2(
        parental_effect_log_likelihood, start=list(beta_m_0=beta_m_0_ini, beta_m_1=beta_m_1_ini, beta_p_0=beta_p_0_ini, beta_p_1=beta_p_1_ini), 
        lower=c(1e-8,1e-8,1e-8,1e-8), upper=c(Inf,Inf,Inf,Inf), method="L-BFGS-B", data=list(x=df)
        )
    coef_ll <- coef(summary(m_ll))
    beta_m_0_ll_est <- coef_ll[[1]]
    beta_m_1_ll_est <- coef_ll[[2]]
    beta_p_0_ll_est <- coef_ll[[3]]
    beta_p_1_ll_est <- coef_ll[[4]]

    m_p = lm(paternal_count ~ paternal_age, data=df)
    m_m = lm(maternal_count ~ maternal_age, data=df)
    beta_m_0_ols_est <- coef(summary(m_m))[[1]]
    beta_m_1_ols_est <- coef(summary(m_m))[[2]]
    beta_p_0_ols_est <- coef(summary(m_p))[[1]]
    beta_p_1_ols_est <- coef(summary(m_p))[[2]]

    intercept_sum = beta_m_0_ll_est + beta_p_0_ll_est
    slope_sum = beta_m_1_ll_est + beta_p_1_ll_est
    cat("total intercept: ", intercept_sum, ", total slope: ", slope_sum, "\n")

    if (print_output) {
        cat("maternal effect (intercept) estimate log likelihood: ",  beta_m_0_ll_est, "OLS: ", beta_m_0_ols_est, "\n")
        cat("paternal effect (intercept) estimate log likelihood: ",  beta_p_0_ll_est, "OLS: ", beta_p_0_ols_est, "\n")
        cat("maternal effect (slope) estimate log likelihood: ",  beta_m_1_ll_est, "OLS: ", beta_m_1_ols_est, "\n")
        cat("paternal effect (slope) estimate log likelihood: ",  beta_p_1_ll_est, "OLS: ", beta_p_1_ols_est, "\n")
        cat("paternal / maternal ratio: ", beta_p_1_ll_est / beta_m_1_ll_est, "OLS: ", beta_p_1_ols_est / beta_m_1_ols_est, "\n")
    }
    return(c(beta_m_1_ll_est, beta_p_1_ll_est, beta_m_1_ols_est, beta_p_1_ols_est, m_ll)) #, beta_m_0_ll_est, beta_p_0_ll_est, beta_m_0_ols_est, beta_p_0_ols_est))
}

compare_ols_vs_ll_estimates <- function(data_generating_function) {
    plot_data = data.frame()

    for (i in 1:100) {
        df_data <- data_generating_function()
        estimates <- estimate_paternal_maternal_effects(df_data, 2, 0.5, 5, 2)
        row <- data.frame(m_ll = estimates[1], p_ll = estimates[2], m_ols = estimates[3], p_ols = estimates[4])
        plot_data <- rbind(plot_data, row)
    }
    p1 <- ggplot(plot_data, aes(x = m_ll, y = m_ols)) + geom_point() 
    p2 <- ggplot(plot_data, aes(x = p_ll, y = p_ols)) + geom_point() 

    grid.arrange(p1, p2, ncol=2)
}

In [5]:
df_ayeaye_wang <- read.csv("./output/phased_counts/ayeaye_wang_phased_counts.csv")
df_ayeaye_wang$paternal_age <- df_ayeaye_wang$paternal_age - 3
df_ayeaye_wang$maternal_age <- df_ayeaye_wang$maternal_age - 3
ayeaye_output <- estimate_paternal_maternal_effects(df_ayeaye_wang, 1, 1, 8, 0.1, print_output=TRUE)

Warning message in mle2(parental_effect_log_likelihood, start = list(beta_m_0 = beta_m_0_ini, :
“some parameters are on the boundary: variance-covariance calculations based on Hessian may be unreliable”


total intercept:  18.04446 , total slope:  3.13757 
maternal effect (intercept) estimate log likelihood:  1e-08 OLS:  -4.720664 
paternal effect (intercept) estimate log likelihood:  18.04446 OLS:  7.789456 
maternal effect (slope) estimate log likelihood:  2.45706 OLS:  1.145687 
paternal effect (slope) estimate log likelihood:  0.6805108 OLS:  0.1145923 
paternal / maternal ratio:  0.2769615 OLS:  0.1000206 


## Baboon

In [6]:
df_baboon <- read.csv("./output/phased_counts/baboon_wang_phased_counts.csv")
df_baboon$paternal_age <- df_baboon$paternal_age - 5.4
df_baboon$maternal_age <- df_baboon$maternal_age - 5.4
baboon_output <- estimate_paternal_maternal_effects(df_baboon, 1.5, 0.5, 6, 0.5, print_output=TRUE)


Warning message in mle2(parental_effect_log_likelihood, start = list(beta_m_0 = beta_m_0_ini, :
“some parameters are on the boundary: variance-covariance calculations based on Hessian may be unreliable”


total intercept:  18.39785 , total slope:  0.9241126 
maternal effect (intercept) estimate log likelihood:  4.463328 OLS:  3.533666 
paternal effect (intercept) estimate log likelihood:  13.93453 OLS:  9.375522 
maternal effect (slope) estimate log likelihood:  1e-08 OLS:  -0.3408031 
paternal effect (slope) estimate log likelihood:  0.9241126 OLS:  0.01835059 
paternal / maternal ratio:  92411263 OLS:  -0.05384513 


## Cat

In [7]:
df_cat <- read.csv("./output/phased_counts/cat_phased_counts.csv")
df_cat$paternal_age <- df_cat$paternal_age - 0.5
df_cat$maternal_age <- df_cat$maternal_age - 0.5
cat_output <- estimate_paternal_maternal_effects(df_cat, 1.5, 0.5, 6, 0.5, print_output=TRUE)


total intercept:  14.26869 , total slope:  1.63562 
maternal effect (intercept) estimate log likelihood:  4.880136 OLS:  1.473285 
paternal effect (intercept) estimate log likelihood:  9.388553 OLS:  6.085556 
maternal effect (slope) estimate log likelihood:  0.003037378 OLS:  0.5090104 
paternal effect (slope) estimate log likelihood:  1.632583 OLS:  0.673312 
paternal / maternal ratio:  537.4973 OLS:  1.322786 


## Chimp

In [8]:
df_chimp <- data.frame(total=c(45, 57, 18, 42, 18, 28, 55),
    paternal_count=c(11, 17, 7, 16, 3, 10, 19),
    maternal_count=c(5, 5, 1, 1, 1, 1, 5), 
    paternal_age=c(21, 23.9, 14.7, 20.22, 15.63, 18.39, 21.07), 
    maternal_age=c(21, 15.89, 14.79, 12.22, 15.72, 18.48, 13.06)
    )
df_chimp$unphased_count <- df_chimp$total - df_chimp$paternal_count - df_chimp$maternal_count
df_chimp$paternal_age <- df_chimp$paternal_age - 13
df_chimp$maternal_age <- df_chimp$maternal_age - 12

chimp_output <- estimate_paternal_maternal_effects(df_chimp, 1, 1, 1, 1, print_output=TRUE)

total intercept:  10.06807 , total slope:  4.478099 
maternal effect (intercept) estimate log likelihood:  5.882468 OLS:  2.068739 
paternal effect (intercept) estimate log likelihood:  4.185606 OLS:  2.450709 
maternal effect (slope) estimate log likelihood:  0.2456793 OLS:  0.166378 
paternal effect (slope) estimate log likelihood:  4.23242 OLS:  1.499545 
paternal / maternal ratio:  17.22741 OLS:  9.012885 


## Human

In [9]:
df_human <- read.csv("./output/phased_counts/human_phased_counts.csv")
df_human$paternal_age <- df_human$paternal_age - 14
df_human$maternal_age <- df_human$maternal_age - 14 
human_output <- estimate_paternal_maternal_effects(df_human, 2, 0.39, 5.5, 1.41, print_output=TRUE)


total intercept:  37.0213 , total slope:  1.934757 
maternal effect (intercept) estimate log likelihood:  8.556946 OLS:  3.277745 
paternal effect (intercept) estimate log likelihood:  28.46435 OLS:  11.32291 
maternal effect (slope) estimate log likelihood:  0.4185496 OLS:  0.1718944 
paternal effect (slope) estimate log likelihood:  1.516207 OLS:  0.5947766 
paternal / maternal ratio:  3.622527 OLS:  3.460128 


## Macaque

In [10]:
df_macaque_wang <- read.csv("./output/phased_counts/macaque_wang_phased_counts.csv")
df_macaque_wang$paternal_age <- df_macaque_wang$paternal_age - 2.5
df_macaque_wang$maternal_age <- df_macaque_wang$maternal_age - 2.5 
macaque_wang_output <- estimate_paternal_maternal_effects(df_macaque_wang, 1, 0.5, 1, 1, print_output=TRUE)

total intercept:  15.05112 , total slope:  1.918352 
maternal effect (intercept) estimate log likelihood:  6.065233 OLS:  1.379759 
paternal effect (intercept) estimate log likelihood:  8.98589 OLS:  0.9457353 
maternal effect (slope) estimate log likelihood:  0.2539662 OLS:  0.2111631 
paternal effect (slope) estimate log likelihood:  1.664385 OLS:  1.24677 
paternal / maternal ratio:  6.553571 OLS:  5.904299 


## Mouse

In [11]:
df_mouse <- read.csv("./output/phased_counts/mouse_phased_counts.csv")
df_mouse$paternal_age <- df_mouse$paternal_age - 0.15
df_mouse$maternal_age <- df_mouse$maternal_age - 0.15 
mouse_output <- estimate_paternal_maternal_effects(df_mouse, 1, 1, 8, 0.1, print_output=TRUE)

Warning message in mle2(parental_effect_log_likelihood, start = list(beta_m_0 = beta_m_0_ini, :
“some parameters are on the boundary: variance-covariance calculations based on Hessian may be unreliable”


total intercept:  18.15763 , total slope:  6.618493 
maternal effect (intercept) estimate log likelihood:  5.745781 OLS:  1.322091 
paternal effect (intercept) estimate log likelihood:  12.41185 OLS:  2.787753 
maternal effect (slope) estimate log likelihood:  1e-08 OLS:  0.6660575 
paternal effect (slope) estimate log likelihood:  6.618493 OLS:  3.652758 
paternal / maternal ratio:  661849328 OLS:  5.484148 


## Owl monkey

In [12]:
df_owl_monkey <- read.csv("./output/phased_counts/owl_monkey_phased_counts.csv")
df_owl_monkey$paternal_age <- df_owl_monkey$paternal_age - 1
df_owl_monkey$maternal_age <- df_owl_monkey$maternal_age - 1
owl_monkey_output <- estimate_paternal_maternal_effects(df_owl_monkey, 1, 1, 8, 0.1, print_output=TRUE)


Warning message in mle2(parental_effect_log_likelihood, start = list(beta_m_0 = beta_m_0_ini, :
“some parameters are on the boundary: variance-covariance calculations based on Hessian may be unreliable”


total intercept:  9.921826 , total slope:  1.824674 
maternal effect (intercept) estimate log likelihood:  5.731455 OLS:  6.265086 
paternal effect (intercept) estimate log likelihood:  4.190371 OLS:  8.512939 
maternal effect (slope) estimate log likelihood:  1e-08 OLS:  -0.6934057 
paternal effect (slope) estimate log likelihood:  1.824674 OLS:  -0.6101195 
paternal / maternal ratio:  182467405 OLS:  0.8798883 
